In [1]:
import pandas as pd
import requests
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Clean DataFrames
def cleanDF(df):
    """
    Input: Pandas DataFrame
    Output: Cleaned Pandas DataFrame
    """
    all_cols = ['ID', 'Language', 'Subscription Type', 'Subscription Event Type',
                'Purchase Store', 'Purchase Amount', 'Currency',
                'Subscription Start Date', 'Subscription Expiration', 'Demo User',
                'Free Trial User', 'Free Trial Start Date', 'Free Trial Expiration',
                'Auto Renew', 'Country', 'User Type', 'Lead Platform',
                'Email Subscriber', 'Push Notifications', 'Send Count', 'Open Count',
                'Click Count', 'Unique Open Count', 'Unique Click Count',
                'App Session Platform', 'App Activity Type', 'App Session Date']

    category_cols = ['Language', 'Subscription Type', 'Subscription Event Type', 'Purchase Store',
                    'Currency', 'Country', 'User Type', 'Lead Platform',
                    'App Session Platform', 'App Activity Type']

    int_cols = ['ID', 'Send Count', 'Open Count', 'Click Count', 'Unique Open Count', 'Unique Click Count']

    date_cols = ['Subscription Start Date', 'Subscription Expiration',
                'App Session Date', 'Free Trial Start Date', 'Free Trial Expiration']

    bool_cols = ['Demo User', 'Free Trial User', 'Auto Renew',
                'Email Subscriber', 'Push Notifications']

    # Keep only columns that exist in df
    all_cols = [col for col in all_cols if col in df.columns]
    category_cols = [col for col in category_cols if col in df.columns]
    int_cols = [col for col in int_cols if col in df.columns]
    date_cols = [col for col in date_cols if col in df.columns]
    bool_cols = [col for col in bool_cols if col in df.columns]

    # if 'Auto Renew' is in df.columns
    if 'Auto Renew' in df.columns:
        # convert to True/False
        df['Auto Renew'] = df['Auto Renew'].replace({'On': True, 'Off': False})

    # convert na to 0 in int_cols
    df[int_cols] = df[int_cols].fillna(0)

    # df[category_cols] = df[category_cols].astype('category')
    df[int_cols] = df[int_cols].astype('int64')
    df[date_cols] = df[date_cols].astype('datetime64[ns]')
    # df[bool_cols] = df[bool_cols].astype('bool')
    # convert bool cols to 1 or 0
    df[bool_cols] = df[bool_cols].astype('int64')
    

    return df

# Fix the Purchase Amounts with the extra 0s at the end 
def transform_value(value):
    if pd.isnull(value):
        return value
    value_str = str(int(value))
    if value_str.endswith('0000'):
        value_int = int(value / 10000)
        return float(value_int / 100)
    else:
        return value

#fetch exchange rates for the currency 
def fetch_exchange_rates(api_key, currencies):
    currency_list = ','.join(currencies)
    url = f'https://openexchangerates.org/api/latest.json?app_id={api_key}&symbols={currency_list}'
    response = requests.get(url)
    
    if response.status_code == 200:
        return response.json()['rates']
    else:
        raise ValueError(f"Failed to fetch exchange rates. Status code: {response.status_code}")

#convert to USD
def convert_to_usd(amount, currency, exchange_rates):
    if currency in exchange_rates:
        return amount / exchange_rates[currency]
    else:
        return amount

# Convert currencies to USD
def convert_currencies_to_usd(data_frame, amount_column, currency_column, exchange_rates):
    data_frame[amount_column] = data_frame.apply(lambda row: convert_to_usd(row[amount_column], row[currency_column], exchange_rates), axis=1)
    return data_frame


In [ ]:

# CREATE MERGED DF

# clean subscriberDF 
subscriberDF = pd.read_csv('../data/Original_Subscriber_Information.csv')
subscriberDF['Purchase Amount'] = subscriberDF['Purchase Amount'].apply(transform_value)

# List of all currencies
currencies = [
    'AED', 'AUD', 'BGN', 'BRL', 'CAD', 'CHF', 'CLP', 'COP', 'CRC', 'CZK',
    'DKK', 'EGP', 'EUR', 'GBP', 'GHS', 'HKD', 'HUF', 'IDR', 'ILS', 'INR',
    'JPY', 'KRW', 'KZT', 'LBP', 'MXN', 'MYR', 'NOK', 'NZD', 'PEN', 'PHP',
    'PLN', 'QAR', 'RON', 'RSD', 'RUB', 'SAR', 'SEK', 'SGD', 'THB', 'TRY',
    'UAH', 'VND', 'ZAR'
]

api_key = '418dd91a76a743c7bee44a1feb77fe51'
exchange_rates = fetch_exchange_rates(api_key, currencies)

subscriberDF = convert_currencies_to_usd(subscriberDF, 'Purchase Amount', 'Currency', exchange_rates)


# Drop rows with null values in the App Activity data
appDF = pd.read_csv('../data/Original_App_activity.csv')
appDF.dropna()
appDF = appDF.dropna(subset=['App Activity Type', 'App Session Platform', 'App Session Date'])

# appDF.to_csv('../data/app_nonull.csv', index=False)                   # save to csv



# Use currencies to determine the country of the subscriber
# this is a dictionary of currency to country
currency_to_country = {
    'AED': 'United Arab Emirates',
    'AUD': 'Australia',
    'BGN': 'Bulgaria',
    'BRL': 'Brazil',
    'CAD': 'Canada',
    'CHF': 'Switzerland',
    'CLP': 'Chile',
    'COP': 'Colombia',
    'CRC': 'Costa Rica',
    'CZK': 'Czech Republic',
    'DKK': 'Denmark',
    'EGP': 'Egypt',
    'EUR': 'European Union',
    'GBP': 'United Kingdom',
    'GHS': 'Ghana',
    'HKD': 'Hong Kong',
    'HUF': 'Hungary',
    'IDR': 'Indonesia',
    'ILS': 'Israel',
    'INR': 'India',
    'JPY': 'Japan',
    'KRW': 'South Korea',
    'KZT': 'Kazakhstan',
    'LBP': 'Lebanon',
    'MXN': 'Mexico',
    'MYR': 'Malaysia',
    'NOK': 'Norway',
    'NZD': 'New Zealand',
    'PEN': 'Peru',
    'PHP': 'Philippines',
    'PLN': 'Poland',
    'QAR': 'Qatar',
    'RON': 'Romania',
    'RSD': 'Serbia',
    'RUB': 'Russia',
    'SAR': 'Saudi Arabia',
    'SEK': 'Sweden',
    'SGD': 'Singapore',
    'THB': 'Thailand',
    'TRY': 'Turkey',
    'UAH': 'Ukraine',
    'USD': 'United States',
    'VND': 'Vietnam',
    'ZAR': 'South Africa'
}

#skip rows with na or nan in currency
subscriberDF['Country'] = subscriberDF['Currency'].apply(lambda currency: currency_to_country[currency] if currency in currency_to_country else None)

# # subscriberDF.to_csv('../data/subscriberClean.csv', index=False)     # save to csv


# This is how we merge the two dataframes
merged_df = subscriberDF.merge(appDF, on='ID', how='left')


# Aggregate the app data by ID
app_agg = appDF.pivot_table(index='ID', columns='App Activity Type', values='App Session Platform', aggfunc='count', fill_value=0)
app_agg = app_agg.join(appDF.pivot_table(index='ID', columns='App Session Platform', values='App Activity Type', aggfunc='count', fill_value=0), rsuffix='_App_Session_Platform')
app_agg = app_agg.join(appDF.groupby('ID').agg({'App Session Date': ['min', 'max']}))

# Rename the columns
app_agg.rename(columns={'android': 'Android Uses'}, inplace=True)
app_agg.rename(columns={'ios': 'iOS Uses'}, inplace=True)
app_agg.rename(columns={'web': 'Web Uses'}, inplace=True)

app_agg.rename(columns={('App Session Date', 'min'): 'First App Session'}, inplace=True)
app_agg.rename(columns={('App Session Date', 'max'): 'Last App Session'}, inplace=True)

# drop repead ID's 
subscriberDF = subscriberDF.drop_duplicates(subset=['ID'])

# merge the two dataframes
aggregatedDF = subscriberDF.merge(app_agg, on='ID', how='left')


# Churn criteria: Subscription expiration date is before March 31st, 2020, or
# (auto-renew is off and the count of non-empty App Activity Types is less than a threshold, e.g., 5)
def determine_churn(row):
    threshold = 4
    churned = (pd.to_datetime(row['Subscription Expiration']) < pd.Timestamp('2020-03-31') or
               (row['Auto Renew'] == 0 and (row['Android Uses'] + row['iOS Uses'] + row['Web Uses']) < threshold))
    return int(churned)

aggregatedDF['Churn'] = aggregatedDF.apply(determine_churn, axis=1)


## save to csv
# aggregatedDF.to_csv('../data/aggregatedClean.csv', index=False)
# merged_df.to_csv("../data/mergedClean.csv")


In [24]:
features = aggregatedDF.columns
features = features.drop(['ID', 'Subscription Expiration', 'Currency', 'Auto Renew', 'First App Session', 'Last App Session'])

# z scoring
z = StandardScaler()
continuous = ['Purchase Amount', 'Send Count', 'Open Count',
              'Click Count', 'Unique Open Count', 'Unique Click Count', 'Android Uses', 'iOS Uses', 'Web Uses']

z_aggregatedDF = aggregatedDF
z_aggregatedDF[continuous] = z.fit_transform(aggregatedDF[continuous])

# dummying
categorical = ['Language', 'Subscription Type', 'Subscription Event Type', 'Purchase Store',
               'Country', 'User Type', 'Lead Platform']

aggDF_dummies = pd.get_dummies(aggregatedDF[categorical])
col_names = [str(col) for col in aggDF_dummies.columns]
# features.append(col_names)

# concat, print head and verify changes
oh_aggDF = aggregatedDF
oh_aggDF = oh_aggDF.drop(categorical, axis=1)
oh_aggDF = pd.concat([z_aggregatedDF, aggDF_dummies], axis=1)

oh_aggDF = oh_aggDF.drop(['ID', 'Subscription Expiration', 'Currency', 'First App Session',
                          'Last App Session', 'Subscription Type_Limited', 'Language_ALL',
                          'Subscription Event Type_RENEWAL', 'Purchase Store_Web', 'Country_Australia',
                          'Lead Platform_Unknown', 'User Type_Other'], axis=1)

#######################################################################################
aggDF = pd.read_csv('../data/aggregatedClean.csv')
X = aggDF.drop(['ID', 'Currency', 
                # 'Purchase Amount', 
                'Subscription Start Date', 'Subscription Expiration', 
                'Free Trial Start Date', 'Free Trial Expiration', 'First App Session', 
                'Last App Session'], axis=1)
# y = aggDF['Purchase Amount']

# sum ['Android Uses', 'iOS Uses', 'Web Uses'] to 'Total Uses'
X['Total Uses'] = X['Android Uses'] + X['iOS Uses'] + X['Web Uses']
X = X.drop(['Android Uses', 'iOS Uses', 'Web Uses'], axis=1)

# bool_cols = columns where nunique() == 2
bool_cols = [col for col in X.columns if X[col].nunique() == 2]

numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns
numeric_cols = numeric_cols.drop(['Churn'])

categorical_cols = X.select_dtypes(include=['object']).columns
categorical_cols = categorical_cols.append(pd.Index(['Churn']))
categorical_cols = categorical_cols.drop(list(bool_cols) + ['Churn'])


# convert bool_cols to int
bool_cols = X.select_dtypes(include=['bool']).columns
X[bool_cols] = X[bool_cols].astype(int)

# drop rows with missing values
X = X.dropna()

# z-score normalization
scaler = StandardScaler()
X[numeric_cols] = scaler.fit_transform(X[numeric_cols])

# one hot encodeing
X = pd.get_dummies(X)

#######################################################################################

#PCA 

# PCA for Click Count
from plotnine import *
from sklearn.decomposition import PCA
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import numpy as np

pca = PCA() 

pca.fit(X)

print(len(pca.explained_variance_ratio_))
print(len(pca.explained_variance_ratio_.cumsum()))

pcaDF = pd.DataFrame({"Column": X.columns,
                      "expl_var" :
                      pca.explained_variance_ratio_,
                      "pc": range(0,len(pca.explained_variance_ratio_)),
                      "cum_var":
                      pca.explained_variance_ratio_.cumsum()})

print(pcaDF)


Index(['Language', 'Subscription Type', 'Subscription Event Type',
       'Purchase Store', 'Purchase Amount', 'Demo User', 'Free Trial User',
       'Auto Renew', 'Country', 'User Type', 'Lead Platform',
       'Email Subscriber', 'Push Notifications', 'Send Count', 'Open Count',
       'Click Count', 'Unique Open Count', 'Unique Click Count', 'App Launch',
       'Completed', 'Onboarding', 'Other', 'Start', 'Android Uses', 'iOS Uses',
       'Web Uses', 'Churn'],
      dtype='object')


KeyError: "['Android Uses', 'iOS Uses', 'Web Uses'] not in index"

# stuff

In [10]:
aggregatedDF.dtypes

ID                           int64
Language                    object
Subscription Type           object
Subscription Event Type     object
Purchase Store              object
Purchase Amount            float64
Currency                    object
Subscription Start Date     object
Subscription Expiration     object
Demo User                   object
Free Trial User             object
Free Trial Start Date       object
Free Trial Expiration       object
Auto Renew                  object
Country                     object
User Type                   object
Lead Platform               object
Email Subscriber            object
Push Notifications          object
Send Count                 float64
Open Count                 float64
Click Count                float64
Unique Open Count          float64
Unique Click Count         float64
App Launch                 float64
Completed                  float64
Onboarding                 float64
Other                      float64
Start               

In [17]:
# z scoring
z = StandardScaler()
continuous = ['Purchase Amount', 'Send Count', 'Open Count',
              'Click Count', 'Unique Open Count', 'Unique Click Count', 'Android Uses', 'iOS Uses', 'Web Uses']

z_aggregatedDF = aggregatedDF
z_aggregatedDF[continuous] = z.fit_transform(aggregatedDF[continuous])

# dummying
categorical = ['Language', 'Subscription Type', 'Subscription Event Type', 'Purchase Store',
               'Country', 'User Type', 'Lead Platform']

aggDF_dummies = pd.get_dummies(aggregatedDF[categorical])
col_names = [str(col) for col in aggDF_dummies.columns]
# features.append(col_names)

# concat, print head and verify changes
oh_aggDF = aggregatedDF
oh_aggDF = oh_aggDF.drop(categorical, axis=1)
oh_aggDF = pd.concat([z_aggregatedDF, aggDF_dummies], axis=1)

oh_aggDF = oh_aggDF.drop(['ID', 'Subscription Expiration', 'Currency', 'First App Session',
                          'Last App Session', 'Subscription Type_Limited', 'Language_ALL',
                          'Subscription Event Type_RENEWAL', 'Purchase Store_Web', 'Country_Australia',
                          'Lead Platform_Unknown', 'User Type_Other'], axis=1)
oh_aggDF.head()



,Language,Subscription Type,Subscription Event Type,Purchase Store,Purchase Amount,Subscription Start Date,Demo User,Free Trial User,Free Trial Start Date,Free Trial Expiration,Auto Renew,Country,User Type,Lead Platform,Email Subscriber,Push Notifications,Send Count,Open Count,Click Count,Unique Open Count,Unique Click Count,App Launch,Completed,Onboarding,Other,Start,Android Uses,iOS Uses,Web Uses,Churn,Language_ARA,Language_CHI,Language_DAR,Language_DEU,Language_EBR,Language_ENG,Language_ESC,Language_ESP,Language_FAR,Language_FRA,Language_GLE,Language_GRK,Language_HEB,Language_HIN,Language_IND,Language_ITA,Language_JPN,Language_KIS,Language_KOR,Language_LAT,Language_NED,Language_PAS,Language_POL,Language_POR,Language_RUS,Language_SVE,Language_TGL,Language_TUR,Language_URD,Language_VIE,Subscription Type_Lifetime,Subscription Event Type_INITIAL_PURCHASE,Purchase Store_App,Country_Brazil,Country_Bulgaria,Country_Canada,Country_Chile,Country_Colombia,Country_Costa Rica,Country_Czech Republic,Country_Denmark,Country_Egypt,Country_European Union,Country_Ghana,Country_Hong Kong,Country_Hungary,Country_India,Country_Indonesia,Country_Israel,Country_Japan,Country_Kazakhstan,Country_Lebanon,Country_Malaysia,Country_Mexico,Country_New Zealand,Country_Norway,Country_Peru,Country_Philippines,Country_Poland,Country_Qatar,Country_Romania,Country_Russia,Country_Saudi Arabia,Country_Serbia,Country_Singapore,Country_South Africa,Country_South Korea,Country_Sweden,Country_Switzerland,Country_Thailand,Country_Turkey,Country_Ukraine,Country_United Arab Emirates,Country_United Kingdom,Country_United States,Country_Vietnam,User Type_Consumer,Lead Platform_App,Lead Platform_Web
0,POR,Limited,INITIAL_PURCHASE,App,NaN,12/28/2018,Yes,No,NaN,NaN,Off,None,Consumer,App,Yes,Yes,0.524503,-0.033036,-0.073269,0.160340,-0.320277,12.0,2.0,0.0,0.0,0.0,-0.224238,-0.184394,-0.230740,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
1,EBR,Limited,INITIAL_PURCHASE,Web,-0.401753,11/28/2019,No,No,NaN,NaN,Off,United States,Consumer,Web,No,Yes,-0.485343,-0.139884,-0.073269,-0.220035,-0.320277,27.0,16.0,0.0,22.0,12.0,2.117025,-0.459522,0.605222,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1
2,ESP,Limited,INITIAL_PURCHASE,Web,-0.911333,12/31/2018,No,No,NaN,NaN,Off,United States,Consumer,Web,Yes,Yes,-0.536691,-0.220021,-0.073269,-0.296110,-0.320277,39.0,37.0,0.0,0.0,0.0,-0.224238,1.034028,-0.230740,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1
3,KOR,Limited,INITIAL_PURCHASE,App,NaN,11/7/2019,Yes,No,NaN,NaN,Off,None,Consumer,App,Yes,Yes,-0.314183,-0.220021,-0.073269,-0.296110,-0.320277,15.0,6.0,0.0,9.0,8.0,-0.224238,0.287253,-0.230740,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
4,ENG,Limited,INITIAL_PURCHASE,App,NaN,8/13/2019,No,No,NaN,NaN,Off,None,Consumer,Web,Yes,Yes,0.815475,-0.086460,-0.039268,0.084265,0.562368,63.0,21.0,0.0,29.0,38.0,1.600570,1.466372,-0.230740,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
